In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader, random_split

from skimage.transform import resize
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
from google.colab import files

drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%%writefile models.py
"""
Classes for models to be trained.
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

def contraction_block(in_channels: int,
                      mid_channels: int,
                      out_channels: int,
                      kernel_size: int=3,
                      stride: int=1,
                      padding: int=1,
                      pool_factor:int=2) -> nn.Module:
  """
  Creates a constituent Conv block for the encoder section of the ptychoNN model.
  Consists of Conv-Relu-Conv-Relu-Maxpool layers.
  Args:
    in_channels: Input channels to the block
    mid_channels: intermediate channels (ie, output of first conv layer and input channels to the second)
    out_channels: Final number of output channels from the conv block
    kernel_size: Uniform kernel size across both conv layers in the block
    stride: Uniform stride across both conv layers in the block
    padding: Uniform padding across both conv layers in the block
    pool_factor: Kernel size of the square max pool
  Returns:
    nn.Sequential container of modules.
  """
  return nn.Sequential(
      nn.Conv2d(in_channels=in_channels, out_channels=mid_channels, kernel_size=kernel_size, stride=stride, padding=padding),
      nn.BatchNorm2d(mid_channels),
      nn.ReLU(),
      nn.Conv2d(in_channels=mid_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
      nn.BatchNorm2d(out_channels),
      nn.ReLU(),
      nn.MaxPool2d(pool_factor)
  )


def expansion_block(in_channels: int,
                    mid_channels: int,
                    out_channels: int,
                    kernel_size: int=3,
                    stride: int=1,
                    padding: int=1,
                    upsamling_factor:int=2) -> nn.Module:
    """
  Creates a constituent Conv block for the decoder sections of the ptychoNN model.
  Consists of Conv-Relu-Conv-Relu-Upsample layers.
  Args:
    in_channels: Input channels to the block
    mid_channels: intermediate channels (ie, output of first conv layer and input channels to the second)
    out_channels: Final number of output channels from the conv block
    kernel_size: Uniform kernel size across both conv layers in the block
    stride: Uniform stride across both conv layers in the block
    padding: Uniform padding across both conv layers in the block
    upsampling_factor: Scale factor for the upsampling layer
  Returns:
    nn.Sequential container of modules.
  """
    return nn.Sequential(
      nn.Conv2d(in_channels=in_channels, out_channels=mid_channels, kernel_size=kernel_size, stride=stride, padding=padding),
      nn.BatchNorm2d(mid_channels),
      nn.ReLU(),
      nn.Conv2d(in_channels=mid_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
      nn.BatchNorm2d(out_channels),
      nn.ReLU(),
      nn.Upsample(scale_factor=upsamling_factor, mode='bilinear')
      )


class PtychoNNBase(nn.Module):
  """
  Defines the deterministic version of the PtychoNN model
  Attributes:
    nconv: number of feature maps from the first conv layer.
  """
  def __init__(self, nconv: int=32, **kwargs):
    super().__init__(**kwargs)
    self.encoder = nn.Sequential(
        contraction_block(in_channels=1, mid_channels=nconv, out_channels=nconv),
        contraction_block(in_channels=nconv, mid_channels=2*nconv, out_channels=2*nconv),
        contraction_block(in_channels=2*nconv, mid_channels=4*nconv, out_channels=4*nconv)
    )
    self.amplitude_decoder = nn.Sequential(
        expansion_block(in_channels=4*nconv, mid_channels=4*nconv, out_channels=4*nconv),
        expansion_block(in_channels=4*nconv, mid_channels=2*nconv, out_channels=2*nconv),
        expansion_block(in_channels=2*nconv, mid_channels=2*nconv, out_channels=2*nconv),
        nn.Conv2d(in_channels=2*nconv, out_channels=1, kernel_size=3, stride=1, padding=1),
        nn.Sigmoid()
    )
    self.phase_decoder = nn.Sequential(
        expansion_block(in_channels=4*nconv, mid_channels=4*nconv, out_channels=4*nconv),
        expansion_block(in_channels=4*nconv, mid_channels=2*nconv, out_channels=2*nconv),
        expansion_block(in_channels=2*nconv, mid_channels=2*nconv, out_channels=2*nconv),
        nn.Conv2d(in_channels=2*nconv, out_channels=1, kernel_size=3, stride=1, padding=1),
        nn.Tanh()
    )

  def forward(self, x):
    encoded = self.encoder(x)
    amps = self.amplitude_decoder(encoded)
    phis = self.phase_decoder(encoded)
    phis = phis * np.pi
    return amps, phis




class PtychoNN(nn.Module):
  """
  Defines the deterministic version of the PtychoNN model
  Attributes:
    nconv: number of feature maps from the first conv layer.
  """
  def __init__(self, nconv: int=32, **kwargs):
    super().__init__(**kwargs)
    self.encoder = nn.Sequential(
        contraction_block(in_channels=1, mid_channels=nconv, out_channels=nconv),
        contraction_block(in_channels=nconv, mid_channels=2*nconv, out_channels=2*nconv),
        contraction_block(in_channels=2*nconv, mid_channels=4*nconv, out_channels=4*nconv)
    )
    self.amplitude_decoder = nn.Sequential(
        expansion_block(in_channels=4*nconv, mid_channels=4*nconv, out_channels=4*nconv),
        expansion_block(in_channels=4*nconv, mid_channels=2*nconv, out_channels=2*nconv),
        expansion_block(in_channels=2*nconv, mid_channels=2*nconv, out_channels=2*nconv),
        nn.Conv2d(in_channels=2*nconv, out_channels=1, kernel_size=3, stride=1, padding=1),
        nn.Sigmoid()
    )
    self.phase_decoder = nn.Sequential(
        expansion_block(in_channels=4*nconv, mid_channels=4*nconv, out_channels=4*nconv),
        expansion_block(in_channels=4*nconv, mid_channels=2*nconv, out_channels=2*nconv),
        expansion_block(in_channels=2*nconv, mid_channels=2*nconv, out_channels=2*nconv),
        nn.Conv2d(in_channels=2*nconv, out_channels=1, kernel_size=3, stride=1, padding=1),
        nn.Tanh()
    )

  def forward(self, x):
    encoded = self.encoder(x)
    amps = self.amplitude_decoder(encoded)
    phis = self.phase_decoder(encoded)
    phis = phis * np.pi
    return amps, phis

  def train_step(self, ft_images, amps, phis):
    pred_amps, pred_phis = self(ft_images)
    amp_loss = F.mse_loss(pred_amps, amps)
    phi_loss = F.mse_loss(pred_phis, phis)
    amp_metric = F.l1_loss(pred_amps, amps)
    phi_metric = F.l1_loss(pred_phis, phis)

    return amp_loss, phi_loss, amp_metric, phi_metric

  def eval_step(self, ft_images, amps, phis):
    pred_amps, pred_phis = self(ft_images)
    amp_loss = F.mse_loss(pred_amps, amps)
    phi_loss = F.mse_loss(pred_phis, phis)
    amp_metric = F.l1_loss(pred_amps, amps)
    phi_metric = F.l1_loss(pred_phis, phis)

    return amp_loss, phi_loss, amp_metric, phi_metric




class PtychoPNN(nn.Module):
  """
  Defines the Probabilistic Neural Network avatar of the PtychoNN model,
  accounting for epistemic uncertainty in predictions.
  The loss function is a NLL loss and the metric is an MSE.
  Attributes:
    nconv: number of feature maps from the first conv layer.
  """
  def __init__(self, nconv: int=32, **kwargs):
    super().__init__(**kwargs)
    self.encoder = nn.Sequential(
        contraction_block(in_channels=1, mid_channels=nconv, out_channels=nconv),
        contraction_block(in_channels=nconv, mid_channels=2*nconv, out_channels=2*nconv),
        contraction_block(in_channels=2*nconv, mid_channels=4*nconv, out_channels=4*nconv)
    )
    self.amplitude_decoder = nn.Sequential(
        expansion_block(in_channels=4*nconv, mid_channels=4*nconv, out_channels=4*nconv),
        expansion_block(in_channels=4*nconv, mid_channels=2*nconv, out_channels=2*nconv),
        expansion_block(in_channels=2*nconv, mid_channels=2*nconv, out_channels=2*nconv),
    )
    self.amplitude_mean_end = nn.Sequential(
        nn.Conv2d(in_channels=2*nconv, out_channels=1, kernel_size=3, stride=1, padding=1),
        nn.Sigmoid()
    )
    self.amplitude_log_sigma = nn.Sequential(
        nn.Conv2d(in_channels=2*nconv, out_channels=1, kernel_size=3, stride=1, padding=1)
    )

    self.phase_decoder = nn.Sequential(
        expansion_block(in_channels=4*nconv, mid_channels=4*nconv, out_channels=4*nconv),
        expansion_block(in_channels=4*nconv, mid_channels=2*nconv, out_channels=2*nconv),
        expansion_block(in_channels=2*nconv, mid_channels=2*nconv, out_channels=2*nconv),
    )
    self.phase_mean_end = nn.Sequential(
        nn.Conv2d(in_channels=2*nconv, out_channels=1, kernel_size=3, stride=1, padding=1),
        nn.Tanh()
    )
    self.phase_log_sigma = nn.Sequential(
        nn.Conv2d(in_channels=2*nconv, out_channels=1, kernel_size=3, stride=1, padding=1)
    )

  def forward(self, x):
    encoded = self.encoder(x)
    amps_decoded = self.amplitude_decoder(encoded)
    amps_mean = self.amplitude_mean_end(amps_decoded)
    amps_logsigma = self.amplitude_log_sigma(amps_decoded)
    phis_decoded = self.phase_decoder(encoded)
    phis_mean = self.phase_mean_end(phis_decoded)
    phis_logsigma = self.phase_log_sigma(phis_decoded)
    phis_mean = phis_mean * np.pi
    return amps_mean, amps_logsigma, phis_mean, phis_logsigma

  def train_step(self, ft_images, amps, phis):
    amps_mean, amps_logsigma, phis_mean, phis_logsigma = self(ft_images)

    amp_loss = F.gaussian_nll_loss(amps_mean, amps, amps_logsigma.exp().square()) #input, target, var
    phi_loss = F.gaussian_nll_loss(phis_mean, phis, phis_logsigma.exp().square())
    amp_metric = F.l1_loss(amps_mean, amps)
    phi_metric = F.l1_loss(phis_mean, phis)

    return amp_loss, phi_loss, amp_metric, phi_metric

  def eval_step(self, ft_images, amps, phis):
    amps_mean, amps_logsigma, phis_mean, phis_logsigma = self(ft_images)
    amp_loss = F.gaussian_nll_loss(amps_mean, amps, amps_logsigma.exp().square()) #input, target, var
    phi_loss = F.gaussian_nll_loss(phis_mean, phis, phis_logsigma.exp().square())
    amp_metric = F.l1_loss(amps_mean, amps)
    phi_metric = F.l1_loss(phis_mean, phis)

    return amp_loss, phi_loss, amp_metric, phi_metric

Overwriting models.py


In [4]:
# H, W = 64, 64
# NLINES = 100
# NLTEST = 60
# N_VALID = 805

In [5]:
%%writefile data_setup.py
"""
Classes & Functions to download data, create datasets and dataloaders.
"""
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader, random_split

from skimage.transform import resize
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

def process_data(diffraction_data: str, real_data: str, NLINES: int, H: int, W: int) -> tuple:
  """
  Takes links for the diffraction and real data files, processes them and
  returns numpy arrays for diffraction data (input) and outputs of real-space
  amplitude and phase.
  Args:
    diffraction_data: path to diffraction data file
    real_data: path to real space data file
    NLINES: Number of lines of scanned data to use for train set
    H: Height of images
    W: Width of images
  Returns:
    tuple of (X_train, Y_I_train, Y_phi_train, X_test, Y_I_test, Y_phi_test)
  """
  data_diffr = np.load(diffraction_data)['arr_0']
  real_space = np.load(real_data)
  amp, ph = np.abs(real_space), np.angle(real_space)

  data_diffr_red = np.zeros((data_diffr.shape[0], data_diffr.shape[1], 64, 64), float)
  for i in range(data_diffr.shape[0]):
    for j in range(data_diffr.shape[1]):
      data_diffr_red[i,j] = resize(data_diffr[i,j,32:-32,32:-32],(64,64),preserve_range=True, anti_aliasing=True)
      data_diffr_red[i,j] = np.where(data_diffr_red[i,j]<3,0,data_diffr_red[i,j])

  tst_strt = amp.shape[0]-NLTEST #Where to index from
  X_train = data_diffr_red[:NLINES,:].reshape(-1,H,W)[:,np.newaxis,:,:]
  X_test = data_diffr_red[tst_strt:,tst_strt:].reshape(-1,H,W)[:,np.newaxis,:,:]
  Y_I_train = amp[:NLINES,:].reshape(-1,H,W)[:,np.newaxis,:,:]
  Y_I_test = amp[tst_strt:,tst_strt:].reshape(-1,H,W)[:,np.newaxis,:,:]
  Y_phi_train = ph[:NLINES,:].reshape(-1,H,W)[:,np.newaxis,:,:]
  Y_phi_test = ph[tst_strt:,tst_strt:].reshape(-1,H,W)[:,np.newaxis,:,:]

  X_train, Y_I_train, Y_phi_train = shuffle(X_train, Y_I_train, Y_phi_train, random_state=0)
  return X_train, Y_I_train, Y_phi_train, X_test, Y_I_test, Y_phi_test


def get_dataloaders(data_path, val_num: int=805, batch_size: int=64, num_workers: int=2)->dict:
  """
  Generates train, validation and test dataloaders from the raw numpy files.
  Args:
    data_path: Location of raw numpy files
    val_num: Number of lines of scan to be saved as validation data
    batch_size: uniform batch size
    num_workers: number of data loader worker processes
  Returns:
    Dict of {"train_dl": train_dl, "val_dl": val_dl, "test_dl": test_dl}
  """
  X_train, Y_I_train, Y_phi_train = torch.from_numpy(np.load(data_path+"X_train.npy")).to(torch.float), torch.from_numpy(np.load(data_path+"Y_I_train.npy")), torch.from_numpy(np.load(data_path+"Y_phi_train.npy"))
  X_test, Y_I_test, Y_phi_test = torch.from_numpy(np.load(data_path+"X_test.npy")).to(torch.float), torch.from_numpy(np.load(data_path+"Y_I_test.npy")), torch.from_numpy(np.load(data_path+"Y_phi_test.npy"))
  train_data_init = TensorDataset(X_train, Y_I_train, Y_phi_train)
  test_dataset = TensorDataset(X_test, Y_I_test, Y_phi_test)
  train_dataset, val_dataset = random_split(train_data_init, [X_train.shape[0]-val_num, val_num])

  train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
  val_dl = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
  test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

  return {"train_dl": train_dl, "val_dl": val_dl, "test_dl": test_dl}

Overwriting data_setup.py


In [6]:
%%writefile utils.py
"""
Utility functions for model training and evaluation.
"""
import torch
import os


def get_devices() -> torch.device:
  """
  Returns gpu device if available, else cpu
  """
  return torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


def set_seeds(seed: int = 42):
  """
  Sets torch seeds to ensure reproducability.
  """
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  os.environ["PYTHONHASHSEED"] = str(seed)


def save_model(model_dir: str, model_name: str, model: torch.nn.Module):
  """
  Saves pytorch model in model_dir with model_name.
  Args:
    model_dir: Directory to save model in.
    model_name: name of file to store model.
    model: model to be saved.
  Returns:
    None
  """
  os.makedirs(model_dir, exist_ok=True)
  if not model_name.endswith("pt"):
    model_name += ".pt"
  torch.save(model.state_dict(), os.path.join(model_dir, model_name))


def create_summary_writer(experiment_name: str, model_name: str, extras: str = None):
        # -> torch.utils.tensorboard.SummaryWriter:
  """
  Instantiates and returns a Summary writer for the experiment, that writers to
  runs/experiment_name/model_name/extras
  Args:
    experiment_name: Name of experiment (say, dataset)
    model_name: Name of model used
    extras: Additional details
  Returns:
    SummaryWriter instance for the experiment
  """
  if extras:
    log_dir = os.path.join("runs/", experiment_name, model_name, extras)
  else:
    log_dir = os.path.join("runs/", experiment_name, model_name)
  writer = torch.utils.tensorboard.SummaryWriter(log_dir)
  return writer

Overwriting utils.py


In [7]:
%%writefile engine.py
"""
Functions to train and evaluate model on the image dataset
"""
import torch
import torch.nn as nn


def train_step(model: torch.nn.Module,
               train_dl: torch.utils.data.DataLoader,
               opt: torch.optim.Optimizer,
               device: torch.device
               ) -> dict:
  """
  Performs 1 epoch of training of model on train dataloader,
  returning model loss on the amplitude and phase reconstruction.
  Args:
    model: model too be trained
    train_dl: Dataloader with training data
    loss_fn: Differentiable loss function to be used for gradients
    opt: Optimizer to train model.
    device: Device on which model and data will reside.
  Returns:
      Dict with keys "amp_loss", "phase_loss", "amp_metric", "phase_metric".
  """
  model.train()
  amplitude_loss, phase_loss, amplitude_metric, phase_metric = 0.0, 0.0, 0.0, 0.0
  for ft_images, amps, phis in train_dl:
    ft_images, amps, phis = ft_images.to(device), amps.to(device), phis.to(device)
    # pred_amps, pred_phis = model(ft_images)
    # amp_loss = loss_fn(pred_amps, amps)
    # phi_loss = loss_fn(pred_phis, phis)
    amp_loss, phi_loss, amp_metric, phi_metric = model.train_step(ft_images, amps, phis)
    loss = amp_loss + phi_loss
    opt.zero_grad()
    loss.backward()
    opt.step()

    amplitude_loss += amp_loss.detach().item()
    phase_loss += phi_loss.detach().item()
    amplitude_metric += amp_metric.detach().item()
    phase_metric += phi_metric.detach().item()

  model.eval()
  return {"amp_loss": amplitude_loss/len(train_dl),
          "phase_loss": phase_loss/len(train_dl),
          "amp_metric": amplitude_metric/len(train_dl),
          "phase_metric": phase_metric/len(train_dl)}


def val_step(model: torch.nn.Module,
            val_dl: torch.utils.data.DataLoader,
            device: torch.device
            ) -> dict:
  """
  Performs 1 epoch of evaluation of model on validation dataloader,
  returning model loss on the amplitude and phase reconstruction.
  Args:
    model: model too be trained
    train_dl: Dataloader with training data
    loss_fn: Loss function to be used for gradients
    device: Device on which model and data will reside.
  Returns:
      Dict with keys "total_loss", "amp_loss" and "phase_loss".
  """
  model.eval()
  amplitude_loss, phase_loss, amplitude_metric, phase_metric = 0.0, 0.0, 0.0, 0.0
  with torch.inference_mode():
    for ft_images, amps, phis in val_dl:
      ft_images, amps, phis = ft_images.to(device), amps.to(device), phis.to(device)
      # pred_amps, pred_phis = model(ft_images)
      # amp_loss = loss_fn(pred_amps, amps)
      # phi_loss = loss_fn(pred_phis, phis)
      amp_loss, phi_loss, amp_metric, phi_metric = model.eval_step(ft_images, amps, phis)
      loss = amp_loss + phi_loss

      amplitude_loss += amp_loss.detach().item()
      phase_loss += phi_loss.detach().item()
      amplitude_metric += amp_metric.detach().item()
      phase_metric += phi_metric.detach().item()

  return {"amp_loss": amplitude_loss/len(val_dl),
          "phase_loss": phase_loss/len(val_dl),
          "amp_metric": amplitude_metric/len(val_dl),
          "phase_metric": phase_metric/len(val_dl)}


def train(model: torch.nn.Module,
          train_dl: torch.utils.data.DataLoader,
          val_dl: torch.utils.data.DataLoader,
          opt: torch.optim.Optimizer,
          device: torch.device,
          num_epochs: int) -> dict:
  """
  Performs defined number of epochs of training and evaluation for the model on
  the data loaders, returning the loss history on amplitude and phase reconstruction.
  Args:
    model: model to be trained and evaluated.
    train_dl: Dataloader with training data.
    val_dl: Dataloader with testing data.
    loss_fn: Differentiable loss function to use for gradients.
    opt: Optimizer to tune model params.
    device: Device on which model and eventually data shall reside
    num_epochs: Number of epochs of training
  Returns:
    Dict with history of "total_loss", "amp_loss" and "phase_loss".
  """
  amp_loss_train, phi_loss_train, amp_loss_val, phi_loss_val = [], [], [], []
  amp_metric_train, phi_metric_train, amp_metric_val, phi_metric_val = [], [], [], []
  for epoch in range(num_epochs):
    train_results = train_step(model, train_dl, opt, device)
    val_results = val_step(model, val_dl, device)
    amp_loss_train.append(train_results["amp_loss"])
    phi_loss_train.append(train_results["phase_loss"])
    amp_loss_val.append(val_results["amp_loss"])
    phi_loss_val.append(val_results["phase_loss"])
    amp_metric_train.append(train_results["amp_metric"])
    phi_metric_train.append(train_results["phase_metric"])
    amp_metric_val.append(val_results["amp_metric"])
    phi_metric_val.append(val_results["phase_metric"])
    print(f"Epoch: {epoch+1} Train Amp Loss: {amp_loss_train[-1]:.5f} Train Phi Loss: {phi_loss_train[-1]:.5f} Val Amp Loss: {amp_loss_val[-1]:.5f} Val Phi Loss: {phi_loss_val[-1]:.5f}")
    print(f"Epoch: {epoch+1} Train Amp Metric: {amp_metric_train[-1]:.5f} Train Phi Metric: {phi_metric_train[-1]:.5f} Val Amp Metric: {amp_metric_val[-1]:.5f} Val Phi Metric: {phi_metric_val[-1]:.5f}")

  return {"amp_loss_train": amp_loss_train, "phi_loss_train": phi_loss_train,
          "amp_loss_val": amp_loss_val, "phi_loss_val": phi_loss_val,
          "amp_metric_train": amp_metric_train, "phi_metric_train": phi_metric_train,
          "amp_metric_val": amp_metric_val, "phi_metric_val": phi_metric_val}



Overwriting engine.py


In [8]:
%%writefile train.py
"""
Takes parameters from user; trains, evaluates and saves models on
Coherent Diffraction Imaging Data.
"""
import torch
import torchvision
import torchvision.transforms as transforms
import os
import argparse
from data_setup import get_dataloaders
from models import PtychoNN, PtychoPNN
from engine import train
from utils import get_devices, set_seeds, save_model


parser = argparse.ArgumentParser()
parser.add_argument("--data_path", type=str, default="./gdrive/MyDrive/PtychoNNData/")
parser.add_argument("--num_epochs", type=int, default=75)
parser.add_argument("--lr", type=float, default=0.0001)
parser.add_argument("--batch_size", type=int, default=64)
parser.add_argument("--model", type=str, default="PtychoNN")
args = parser.parse_args()

set_seeds(42)
device = get_devices()
d = get_dataloaders(args.data_path)
train_dl, val_dl, test_dl = d["train_dl"], d["val_dl"], d["test_dl"]
model = PtychoPNN().to(device)
opt = torch.optim.Adam(model.parameters(), lr=args.lr)
results = train(model, train_dl, val_dl, opt, device, args.num_epochs)
model_name = args.model + str(args.num_epochs)
save_model("./Models", "model_" + args.model, model)

Overwriting train.py


In [9]:
!python train.py

Epoch: 1 Train Amp Loss: -2.61867 Train Phi Loss: 0.10528 Val Amp Loss: -3.98941 Val Phi Loss: -0.06045
Epoch: 1 Train Amp Metric: 0.06441 Train Phi Metric: 0.59055 Val Amp Metric: 0.00975 Val Phi Metric: 0.51094
Epoch: 2 Train Amp Loss: -4.16967 Train Phi Loss: -0.16356 Val Amp Loss: -4.24345 Val Phi Loss: -0.21500
Epoch: 2 Train Amp Metric: 0.00798 Train Phi Metric: 0.47110 Val Amp Metric: 0.00716 Val Phi Metric: 0.45258
Epoch: 3 Train Amp Loss: -4.29502 Train Phi Loss: -0.29608 Val Amp Loss: -4.34528 Val Phi Loss: -0.36282
Epoch: 3 Train Amp Metric: 0.00699 Train Phi Metric: 0.42322 Val Amp Metric: 0.00679 Val Phi Metric: 0.40450
Epoch: 4 Train Amp Loss: -4.36022 Train Phi Loss: -0.40452 Val Amp Loss: -4.35053 Val Phi Loss: -0.32901
Epoch: 4 Train Amp Metric: 0.00672 Train Phi Metric: 0.38990 Val Amp Metric: 0.00670 Val Phi Metric: 0.41390
Epoch: 5 Train Amp Loss: -4.40392 Train Phi Loss: -0.47879 Val Amp Loss: -4.41007 Val Phi Loss: -0.48716
Epoch: 5 Train Amp Metric: 0.00656 Train